# CNN for Image Classification

Halo, semuanya! 👋🏻

Selamat datang di Kuliah Praktisi "Jaringan Syaraf Tiruan" atau *Artificial Neural Networks*. Mari kita kenalan dulu!

* Saya **Syahrul Bahar Hamdani**, panggil aja **Dani**
* Matematika 2012, lulus 2016. Ambil ROK, pakai PSO untuk Penjadwalan Meeting di skripsi, dibimbing oleh Pak Herry dan Bu Auli 🙏🏻
* Ambil S2 Sains Komputasi ITB tahun 2017 dan lulus 2019. Ambil tesis berjudul "**_Predictive Maintenance_ Mesin Pesawat dengan Pendekatan _Machine Learning_**" ([bukti](https://digilib.itb.ac.id/index.php/gdl/view/35771)) yang dibimbing oleh Bu Nuning Nuraini
* Sekarang bekerja sebagai **Lead Data Scientist** di [KoinWorks](https://koinworks.com/)

## Agenda

Di notebook ini, kita akan bahas bagaimana membuat model deep learning CNN untuk klasifikasi gambar. Kita akan coba menggunakan _top-down_ approach, dimulai dari hasil akhir model, kemudian kita akan coba bedah komponen penyusunnya.

Agenda kita hari ini:
* CNN dengan PyTorch
* Kenapa menggunakan CNN?
* Bagaimana cara kerja CNN?

In [ ]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision

## Datasets

[Food-101 dataset](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/).

In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch

In [ ]:
DATA_DIR = Path("data")
CIFAR_DIR = DATA_DIR / "cifar-10-batches-py"

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(DATA_DIR, train=True, download=True)
test_dataset = torchvision.datasets.CIFAR10(DATA_DIR, train=False, download=True)

In [ ]:
def load_cifar(is_train=True):
    train_list = [
        "data_batch_1",
        "data_batch_2",
        "data_batch_3",
        "data_batch_4",
        "data_batch_5",
    ]
    test_list = [
        "test_batch",
    ]
    meta = {
        "filename": "batches.meta",
        "key": "label_names",
    }

    if is_train:
        downloaded_list = train_list
    else:
        downloaded_list = test_lit

    data = []
    labels = []

    # load image data
    for file_name in downloaded_list:
        file_path = CIFAR_DIR / file_name
        with open(file_path, "rb") as f:
            entry = pickle.load(f, encoding="latin1")
            data.append(entry["data"])
            if "labels" in entry:
                labels.extend(entry["labels"])
            else:
                labels.extend(entry["fine_labels"])

    data = np.vstack(data).reshape(-1, 3, 32, 32)
    data = data.transpose(0, 2, 3, 1)

    # load metadata
    meta_file_path = CIFAR_DIR / meta["filename"]
    with open(meta_file_path, "rb") as infile:
        metadata = pickle.load(infile, encoding="latin1")
        classes = metadata[meta["key"]]
    class_to_idx = {_class: i for i, _class in enumerate(classes)}

    return data, labels, classes, class_to_idx

In [ ]:
train_data, train_labels, train_classes, train_class2idx = load_cifar(is_train=True)


In [ ]:
def visualize_sample_images(images, labels, class_names):
    rng = np.random.default_rng()
    rand_idx = rng.integers(0, len(train_labels), 25)
    img_samples = images[rand_idx]
    label_samples = [class_names[labels[idx]] for idx in rand_idx]

    fig = plt.figure(figsize=(8, 8), tight_layout=True)
    for ax in range(1, 26):
        fig.add_subplot(5, 5, ax)
        plt.imshow(img_samples[ax-1])
        plt.title(label_samples[ax-1])
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
visualize_sample_images(train_data, train_labels, train_classes)

In [ ]:
df_metadata.head()

In [ ]:
df_metadata.tail()

In [ ]:
list_channels = list(IMAGE_DATA_DIR.glob("*"))
print("Num of channels:", len(list_channels))

for channel in list_channels:
    print("sample video from '{}':".format(channel.name))
    for video in channel.glob("*.jpg"):
        video_id = video.name.split(".")[0]
        print("video name:", video, "category:", df_metadata.loc[df_metadata.Id.isin([video_id]), "Category"].item())
        break